<a href="https://colab.research.google.com/github/panagiotismouts/machinelearning/blob/main/classimbalance/Assignment_1_Advanced_Topics_in_Machine_Learning_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment 1 - Part 2



Andreas Kiziridis - Erasmus Student

Moutsiounas Panagiotis - 153

In this part of the assignment, we will 

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df = pd.read_csv("carclaims.csv")

#print(df[:10])
df['FraudFound'].value_counts()

X = df.iloc[:, :-1]

y = df.iloc[:, -1]

In [2]:
y

0         No
1         No
2         No
3         No
4         No
        ... 
15415    Yes
15416     No
15417    Yes
15418     No
15419    Yes
Name: FraudFound, Length: 15420, dtype: object

In [2]:
from sklearn.preprocessing import LabelEncoder

X_converted = X.copy()

y_converted = y.copy()

# Instantiate the encoder and fit on the categorical columns
encoder = LabelEncoder()

cat_cols = X_converted.select_dtypes(include=['object']).columns.tolist()


for col in cat_cols:
    X_converted[col] = encoder.fit_transform(X_converted[col])

y_converted = encoder.fit_transform(y_converted.values)

print(y_converted)
# 0 means no fraud, 1 means fraud after label encoding

[0 0 0 ... 1 0 1]


By using the value_counts() method we can see that the dataset is very imbalanced. We proceed in using some class imbalance technics.

In [6]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from collections import Counter

X_train, X_test, y_train, y_test = train_test_split(X_converted, y_converted, test_size=0.3, random_state=42)

#training the data
names = ['random forest', 'linear SVM', 'gaussian naïve bayes']

classifiers = [RandomForestClassifier(n_estimators=150, random_state=42), 
               LinearSVC(max_iter= 2000, C=0.7), GaussianNB()]

for name, clf in zip(names, classifiers):
  print(" ")
  print(name)
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(classification_report(y_test, y_pred, target_names=['no fraud', 'fraud']))
  conf_m = confusion_matrix(y_test, y_pred).T # transpose to align with slides
  print(conf_m)


 
random forest
              precision    recall  f1-score   support

    no fraud       0.94      1.00      0.97      4341
       fraud       1.00      0.00      0.01       285

    accuracy                           0.94      4626
   macro avg       0.97      0.50      0.49      4626
weighted avg       0.94      0.94      0.91      4626

[[4341  284]
 [   0    1]]
 
linear SVM
              precision    recall  f1-score   support

    no fraud       0.94      1.00      0.97      4341
       fraud       0.00      0.00      0.00       285

    accuracy                           0.94      4626
   macro avg       0.47      0.50      0.48      4626
weighted avg       0.88      0.94      0.91      4626

[[4341  285]
 [   0    0]]
 
gaussian naïve bayes
              precision    recall  f1-score   support

    no fraud       0.94      0.96      0.95      4341
       fraud       0.18      0.15      0.16       285

    accuracy                           0.91      4626
   macro avg       0.5

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_

In [10]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier

from imblearn.under_sampling import NearMiss

X_train, X_test, y_train, y_test = train_test_split(X_converted, y_converted, test_size=0.3, random_state=42, stratify=y_converted)

#OVERSAMPLING using SMOTE (Synthetic Minority Over-sampling Technique)
pipeline = make_pipeline(RandomForestClassifier(n_estimators=150, random_state=42))
pipeline.fit(X_train, y_train)

print("Random Forest Classifier")
print(classification_report_imbalanced(y_test, pipeline.predict(X_test)))



pipeline = make_pipeline(SMOTE(random_state=3, k_neighbors=5),
                         RandomForestClassifier(n_estimators=150, random_state=42))
pipeline.fit(X_train, y_train)


print("RF with SMOTE")
print(classification_report_imbalanced(y_test, pipeline.predict(X_test)))

#############################
#UNDERSAMPLING using NearMiss

pipeline = make_pipeline(NearMiss(version=1),
                         RandomForestClassifier(n_estimators=150, random_state=42))
pipeline.fit(X_train, y_train)

print("RF with near miss 1")
print(classification_report_imbalanced(y_test, pipeline.predict(X_test)))

#####################################
#Easy Ensemble

ee = EasyEnsembleClassifier(random_state=42)
ee.fit(X_train, y_train)

print("EasyEnsemble")
print(classification_report_imbalanced(y_test, ee.predict(X_test)))

Random Forest Classifier
                   pre       rec       spe        f1       geo       iba       sup

          0       0.94      1.00      0.02      0.97      0.13      0.02      4349
          1       0.83      0.02      1.00      0.04      0.13      0.02       277

avg / total       0.93      0.94      0.08      0.91      0.13      0.02      4626

RF with SMOTE
                   pre       rec       spe        f1       geo       iba       sup

          0       0.95      0.96      0.16      0.95      0.39      0.16      4349
          1       0.19      0.16      0.96      0.17      0.39      0.14       277

avg / total       0.90      0.91      0.20      0.91      0.39      0.16      4626

RF with near miss 1
                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.61      0.92      0.75      0.75      0.54      4349
          1       0.13      0.92      0.61      0.23      0.75      0.58       277

avg / total       0.